In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from factor_analyzer import FactorAnalyzer

In [2]:
data = pd.read_csv('./merged_df_v4.csv')
data

,ID,time,extra_obs,special,type,lat,lon,vmax,mslp,wmo_basin,...,median_windgusts_10m,median_soil_temperature_0_to_7cm,median_soil_temperature_7_to_28cm,median_soil_temperature_28_to_100cm,median_soil_temperature_100_to_255cm,median_soil_moisture_0_to_7cm,median_soil_moisture_7_to_28cm,median_soil_moisture_28_to_100cm,median_soil_moisture_100_to_255cm,target_cost
0,804,9/16/1998 0:00,0,NaN,TD,10.0,-28.1,30,1009.0,north_atlantic,...,43.95,28.4,28.40,28.4,28.4,0.00,0.0,0.00,0.0,11191.0
1,980,9/14/2002 18:00,0,NaN,TD,10.0,-60.5,25,1009.0,north_atlantic,...,22.85,29.4,29.40,29.4,29.4,0.00,0.0,0.00,0.0,1943.5
2,923,9/21/2002 18:00,0,NaN,TD,10.2,-44.6,25,1009.0,north_atlantic,...,21.40,28.9,28.90,28.9,28.9,0.00,0.0,0.00,0.0,1866.6
3,1135,9/5/2004 18:00,0,NaN,HU,10.2,-46.8,110,955.0,north_atlantic,...,47.35,29.9,29.90,29.9,29.9,0.00,0.0,0.00,0.0,33005.8
4,981,9/15/2002 0:00,0,NaN,TD,10.2,-62.4,25,1009.0,north_atlantic,...,24.30,26.6,27.05,27.0,26.7,0.51,0.5,0.49,0.5,1943.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2726,1124,9/3/2004 0:00,0,NaN,TD,9.7,-28.7,30,1007.0,north_atlantic,...,36.40,28.2,28.20,28.2,28.2,0.00,0.0,0.00,0.0,33005.8
2727,1125,9/3/2004 6:00,0,NaN,TS,9.7,-30.3,35,1005.0,north_atlantic,...,44.45,28.1,28.10,28.1,28.1,0.00,0.0,0.00,0.0,33005.8
2728,803,9/15/1998 18:00,0,NaN,TD,9.8,-26.5,30,1009.0,north_atlantic,...,47.50,28.2,28.20,28.2,28.2,0.00,0.0,0.00,0.0,11191.0
2729,1134,9/5/2004 12:00,0,NaN,HU,9.8,-45.1,85,977.0,north_atlantic,...,60.70,29.4,29.40,29.4,29.4,0.00,0.0,0.00,0.0,33005.8


In [3]:
list(data.columns)

['ID',
 'time',
 'extra_obs',
 'special',
 'type',
 'lat',
 'lon',
 'vmax',
 'mslp',
 'wmo_basin',
 'storm_name',
 'year',
 'Name',
 'Disaster',
 'Begin Date',
 'End Date',
 'Deaths',
 'name',
 'elevation',
 'new_elevation',
 'date',
 'time_only',
 'day',
 'month',
 'min_temperature_2m',
 'min_relativehumidity_2m',
 'min_dewpoint_2m',
 'min_apparent_temperature',
 'min_precipitationmin_',
 'min_rain',
 'min_snowfall',
 'min_snow_depth',
 'min_weathercode',
 'min_pressure_msl',
 'min_surface_pressure',
 'min_cloudcover',
 'min_cloudcover_low',
 'min_cloudcover_mid',
 'min_cloudcover_high',
 'min_et0_fao_evapotranspiration',
 'min_vapor_pressure_deficit',
 'min_windspeed_10m',
 'min_windspeed_100m',
 'min_winddirection_10m',
 'min_winddirection_100m',
 'min_windgusts_10m',
 'min_soil_temperature_0_to_7cm',
 'min_soil_temperature_7_to_28cm',
 'min_soil_temperature_28_to_100cm',
 'min_soil_temperature_100_to_255cm',
 'min_soil_moisture_0_to_7cm',
 'min_soil_moisture_7_to_28cm',
 'min_soil_

In [4]:
stormNames = list(data['name'].unique())

In [5]:
rowIDs = list(data.ID)

In [6]:
# directory = '../api_responses'
# rowID = '0'

# file_path = f"{directory}/{rowID}.json"

# file = load_responses_from_file(file_path)

# len(file['hourly_units'].items())
# list(file['hourly_units'].keys())

In [7]:
import json
import os
import tqdm

def load_responses_from_file(file_name):
    try:
        with open(file_name, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"File not found: {file_name}")
        return None  # Return None if file is not found
    except json.JSONDecodeError:
        print(f"Error parsing JSON file: {file_name}")
    
def get_list_mean(data, var):
    values = [x if x is not None else np.nan for x in data['hourly'][var]]
    return round(np.nanmean(np.array(values)), 2)

def get_list_median(data, var):
    values = [x if x is not None else np.nan for x in data['hourly'][var]]
    return round(np.nanmedian(np.array(values)), 2)

def get_list_min(data, var):
    values = [x if x is not None else np.nan for x in data['hourly'][var]]
    return round(np.nanmin(np.array(values)), 2)

def get_list_max(data, var):
    values = [x if x is not None else np.nan for x in data['hourly'][var]]
    return round(np.nanmax(np.array(values)), 2)

In [8]:
import json
import os
import numpy as np
import pandas as pd

def aggregatedata(data):
    remainderList = ["temperature_2m",
                    "relativehumidity_2m",
                    "dewpoint_2m",
                    "apparent_temperature",
                    "precipitation",
                    "rain",
                    "pressure_msl",
                    "surface_pressure",
                    "cloudcover",
                    "cloudcover_low",
                    "cloudcover_mid",
                    "cloudcover_high",
                    "et0_fao_evapotranspiration",
                    "vapor_pressure_deficit",
                    "windspeed_10m",
                    "windspeed_100m",
                    "winddirection_10m",
                    "winddirection_100m",
                    "windgusts_10m",
                    "soil_temperature_0_to_7cm",
                    "soil_temperature_7_to_28cm",
                    "soil_temperature_28_to_100cm",
                    "soil_temperature_100_to_255cm",
                    "soil_moisture_0_to_7cm",
                    "soil_moisture_7_to_28cm",
                    "soil_moisture_28_to_100cm",
                    "soil_moisture_100_to_255cm"]


    aggData = {}

    directory = '../api_responses' 

    for _, storm in data.iterrows():
        dkey = storm['name']  

        if dkey not in aggData:
            aggData[dkey] = {key: [] for key in remainderList}
            aggData[dkey].update({
                "rowID": [],
                "time": [],
                "deaths": [],
                "elevation": [],
                "target_cost": []
            })

        file_path = f"{directory}/{storm['ID']}.json"
        file = load_responses_from_file(file_path)

        if file is None:
            continue

        aggData[dkey]['rowID'].append(storm['ID'])
        aggData[dkey]['time'].append(storm['time'])
        aggData[dkey]['deaths'].append(storm['Deaths']) 
        aggData[dkey]['elevation'].append(file.get('elevation', np.nan))  
        aggData[dkey]['target_cost'].append(storm['target_cost'])


        for n in remainderList:
            hourly_data = file.get('hourly', {}).get(n, [])
            aggData[dkey][n].extend(hourly_data)  

    return aggData

# aggData = aggregatedata(data)

In [9]:
# # save to json 

# import json

# with open('./aggData.json', 'w') as json_file:
#     json.dump(aggData, json_file)

In [10]:
aggData = load_responses_from_file('./aggData.json')


In [11]:
# number of storms
len(list(aggData.keys()))

57

In [12]:
list(aggData['Katrina'].keys())

['temperature_2m',
 'relativehumidity_2m',
 'dewpoint_2m',
 'apparent_temperature',
 'precipitation',
 'rain',
 'pressure_msl',
 'surface_pressure',
 'cloudcover',
 'cloudcover_low',
 'cloudcover_mid',
 'cloudcover_high',
 'et0_fao_evapotranspiration',
 'vapor_pressure_deficit',
 'windspeed_10m',
 'windspeed_100m',
 'winddirection_10m',
 'winddirection_100m',
 'windgusts_10m',
 'soil_temperature_0_to_7cm',
 'soil_temperature_7_to_28cm',
 'soil_temperature_28_to_100cm',
 'soil_temperature_100_to_255cm',
 'soil_moisture_0_to_7cm',
 'soil_moisture_7_to_28cm',
 'soil_moisture_28_to_100cm',
 'soil_moisture_100_to_255cm',
 'rowID',
 'time',
 'deaths',
 'elevation',
 'target_cost']

In [27]:
lstCols = [
        'storm',
        'deaths',
        'min_time',
        'max_time',
        'avg_elevation',
        'min_elevation',
        'max_elevation',
        'med_elevation',

        'avg_temperature_2m',
        'min_temperature_2m',
        'max_temperature_2m',
        'med_temperature_2m',

        'avg_relativehumidity_2m',
        'min_relativehumidity_2m',
        'max_relativehumidity_2m',
        'med_relativehumidity_2m',
        
        'avg_dewpoint_2m',
        'min_dewpoint_2m',
        'max_dewpoint_2m',
        'med_dewpoint_2m',
        
        'avg_apparent_temperature',
        'min_apparent_temperature',
        'max_apparent_temperature',
        'med_apparent_temperature',
        
        'avg_precipitation',
        'min_precipitation',
        'max_precipitation',
        'med_precipitation',
        
        'avg_rain',
        'min_rain',
        'max_rain',
        'med_rain',
        
        'avg_pressure_msl',
        'min_pressure_msl',
        'max_pressure_msl',
        'med_pressure_msl',
        
        'avg_surface_pressure',
        'min_surface_pressure',
        'max_surface_pressure',
        'med_surface_pressure',
        
        'avg_cloudcover',
        'min_cloudcover',
        'max_cloudcover',
        'med_cloudcover',
        
        'avg_cloudcover_low',
        'min_cloudcover_low',
        'max_cloudcover_low',
        'med_cloudcover_low',
        
        'avg_cloudcover_mid',
        'min_cloudcover_mid',
        'max_cloudcover_mid',
        'med_cloudcover_mid',
        
        'avg_cloudcover_high',
        'min_cloudcover_high',
        'max_cloudcover_high',
        'med_cloudcover_high',
        
        'avg_et0_fao_evapotranspiration',
        'min_et0_fao_evapotranspiration',
        'max_et0_fao_evapotranspiration',
        'med_et0_fao_evapotranspiration',
        
        'avg_vapor_pressure_deficit',
        'min_vapor_pressure_deficit',
        'max_vapor_pressure_deficit',
        'med_vapor_pressure_deficit',
        
        'avg_windspeed_10m',
        'min_windspeed_10m',
        'max_windspeed_10m',
        'med_windspeed_10m',
        
        'avg_windspeed_100m',
        'min_windspeed_100m',
        'max_windspeed_100m',
        'med_windspeed_100m',
        
        'avg_winddirection_10m',
        'min_winddirection_10m',
        'max_winddirection_10m',
        'med_winddirection_10m',
        
        'avg_winddirection_100m',
        'min_winddirection_100m',
        'max_winddirection_100m',
        'med_winddirection_100m',
        
        'avg_windgusts_10m',
        'min_windgusts_10m',
        'max_windgusts_10m',
        'med_windgusts_10m',
        
        'avg_soil_temperature_0_to_7cm',
        'min_soil_temperature_0_to_7cm',
        'max_soil_temperature_0_to_7cm',
        'med_soil_temperature_0_to_7cm',
        
        'avg_soil_temperature_7_to_28cm',
        'min_soil_temperature_7_to_28cm',
        'max_soil_temperature_7_to_28cm',
        'med_soil_temperature_7_to_28cm',
        
        'avg_soil_temperature_28_to_100cm',
        'min_soil_temperature_28_to_100cm',
        'max_soil_temperature_28_to_100cm',
        'med_soil_temperature_28_to_100cm',
        
        'avg_soil_temperature_100_to_255cm',
        'min_soil_temperature_100_to_255cm',
        'max_soil_temperature_100_to_255cm',
        'med_soil_temperature_100_to_255cm',
        
        'avg_soil_moisture_0_to_7cm',
        'min_soil_moisture_0_to_7cm',
        'max_soil_moisture_0_to_7cm',
        'med_soil_moisture_0_to_7cm',
        
        'avg_soil_moisture_7_to_28cm',
        'min_soil_moisture_7_to_28cm',
        'max_soil_moisture_7_to_28cm',
        'med_soil_moisture_7_to_28cm',
        
        'avg_soil_moisture_28_to_100cm',
        'min_soil_moisture_28_to_100cm',
        'max_soil_moisture_28_to_100cm',
        'med_soil_moisture_28_to_100cm',
        
        'avg_soil_moisture_100_to_255cm',
        'min_soil_moisture_100_to_255cm',
        'max_soil_moisture_100_to_255cm',
        'med_soil_moisture_100_to_255cm',
                
        'target_cost'
        ]

len(lstCols)

117

In [28]:
from datetime import datetime

def get_list_mean(data, var):
    values = [x if x is not None else np.nan for x in data[var]]
    return round(np.nanmean(np.array(values)), 2)

def get_list_median(data, var):
    values = [x if x is not None else np.nan for x in data[var]]
    return round(np.nanmedian(np.array(values)), 2)

def get_list_min(data, var):
    values = [x if x is not None else np.nan for x in data[var]]
    return round(np.nanmin(np.array(values)), 2)

def get_list_max(data, var):
    values = [x if x is not None else np.nan for x in data[var]]
    return round(np.nanmax(np.array(values)), 2)



def generateNewRow(stormData, stormName):
    
    dates = [datetime.strptime(date_str, '%m/%d/%Y %H:%M').date() for date_str in stormData['time']]
    min_date = min(dates)
    max_date = max(dates)
    newRowData = {
        'storm': stormName,
        'deaths': stormData['deaths'][0],
        'min_time': min_date,
        'max_time':max_date,
        'avg_elevation': get_list_mean(stormData, 'elevation'),
        'min_elevation': get_list_min(stormData, 'elevation'),
        'max_elevation': get_list_max(stormData, 'elevation'),
        'med_elevation': get_list_median(stormData, 'elevation'),

        'avg_temperature_2m': get_list_mean(stormData, 'temperature_2m'),
        'min_temperature_2m': get_list_min(stormData, 'temperature_2m'),
        'max_temperature_2m': get_list_max(stormData, 'temperature_2m'),
        'med_temperature_2m': get_list_median(stormData, 'temperature_2m'),

        'avg_relativehumidity_2m': get_list_mean(stormData, 'relativehumidity_2m'),
        'min_relativehumidity_2m': get_list_min(stormData, 'relativehumidity_2m'),
        'max_relativehumidity_2m': get_list_max(stormData, 'relativehumidity_2m'),
        'med_relativehumidity_2m': get_list_median(stormData, 'relativehumidity_2m'),
        
        'avg_dewpoint_2m': get_list_mean(stormData, 'dewpoint_2m'),
        'min_dewpoint_2m': get_list_min(stormData, 'dewpoint_2m'),
        'max_dewpoint_2m': get_list_max(stormData, 'dewpoint_2m'),
        'med_dewpoint_2m': get_list_median(stormData, 'dewpoint_2m'),
        
        'avg_apparent_temperature': get_list_mean(stormData, 'apparent_temperature'),
        'min_apparent_temperature': get_list_min(stormData, 'apparent_temperature'),
        'max_apparent_temperature': get_list_max(stormData, 'apparent_temperature'),
        'med_apparent_temperature': get_list_median(stormData, 'apparent_temperature'),
        
        'avg_precipitation': get_list_mean(stormData, 'precipitation'),
        'min_precipitation': get_list_min(stormData, 'precipitation'),
        'max_precipitation': get_list_max(stormData, 'precipitation'),
        'med_precipitation': get_list_median(stormData, 'precipitation'),
        
        'avg_rain': get_list_mean(stormData, 'rain'),
        'min_rain': get_list_min(stormData, 'rain'),
        'max_rain': get_list_max(stormData, 'rain'),
        'med_rain': get_list_median(stormData, 'rain'),
        
        'avg_pressure_msl': get_list_mean(stormData, 'pressure_msl'),
        'min_pressure_msl': get_list_min(stormData, 'pressure_msl'),
        'max_pressure_msl': get_list_max(stormData, 'pressure_msl'),
        'med_pressure_msl': get_list_median(stormData, 'pressure_msl'),
        
        'avg_surface_pressure': get_list_mean(stormData, 'surface_pressure'),
        'min_surface_pressure': get_list_min(stormData, 'surface_pressure'),
        'max_surface_pressure': get_list_max(stormData, 'surface_pressure'),
        'med_surface_pressure': get_list_median(stormData, 'surface_pressure'),
        
        'avg_cloudcover': get_list_mean(stormData, 'cloudcover'),
        'min_cloudcover': get_list_min(stormData, 'cloudcover'),
        'max_cloudcover': get_list_max(stormData, 'cloudcover'),
        'med_cloudcover': get_list_median(stormData, 'cloudcover'),
        
        'avg_cloudcover_low': get_list_mean(stormData, 'cloudcover_low'),
        'min_cloudcover_low': get_list_min(stormData, 'cloudcover_low'),
        'max_cloudcover_low': get_list_max(stormData, 'cloudcover_low'),
        'med_cloudcover_low': get_list_median(stormData, 'cloudcover_low'),
        
        'avg_cloudcover_mid': get_list_mean(stormData, 'cloudcover_mid'),
        'min_cloudcover_mid': get_list_min(stormData, 'cloudcover_mid'),
        'max_cloudcover_mid': get_list_max(stormData, 'cloudcover_mid'),
        'med_cloudcover_mid': get_list_median(stormData, 'cloudcover_mid'),
        
        'avg_cloudcover_high': get_list_mean(stormData, 'cloudcover_high'),
        'min_cloudcover_high': get_list_min(stormData, 'cloudcover_high'),
        'max_cloudcover_high': get_list_max(stormData, 'cloudcover_high'),
        'med_cloudcover_high': get_list_median(stormData, 'cloudcover_high'),
        
        'avg_et0_fao_evapotranspiration': get_list_mean(stormData, 'et0_fao_evapotranspiration'),
        'min_et0_fao_evapotranspiration': get_list_min(stormData, 'et0_fao_evapotranspiration'),
        'max_et0_fao_evapotranspiration': get_list_max(stormData, 'et0_fao_evapotranspiration'),
        'med_et0_fao_evapotranspiration': get_list_median(stormData, 'et0_fao_evapotranspiration'),
        
        'avg_vapor_pressure_deficit': get_list_mean(stormData, 'vapor_pressure_deficit'),
        'min_vapor_pressure_deficit': get_list_min(stormData, 'vapor_pressure_deficit'),
        'max_vapor_pressure_deficit': get_list_max(stormData, 'vapor_pressure_deficit'),
        'med_vapor_pressure_deficit': get_list_median(stormData, 'vapor_pressure_deficit'),
        
        'avg_windspeed_10m': get_list_mean(stormData, 'windspeed_10m'),
        'min_windspeed_10m': get_list_min(stormData, 'windspeed_10m'),
        'max_windspeed_10m': get_list_max(stormData, 'windspeed_10m'),
        'med_windspeed_10m': get_list_median(stormData, 'windspeed_10m'),
        
        'avg_windspeed_100m': get_list_mean(stormData, 'windspeed_100m'),
        'min_windspeed_100m': get_list_min(stormData, 'windspeed_100m'),
        'max_windspeed_100m': get_list_max(stormData, 'windspeed_100m'),
        'med_windspeed_100m': get_list_median(stormData, 'windspeed_100m'),
        
        'avg_winddirection_10m': get_list_mean(stormData, 'winddirection_10m'),
        'min_winddirection_10m': get_list_min(stormData, 'winddirection_10m'),
        'max_winddirection_10m': get_list_max(stormData, 'winddirection_10m'),
        'med_winddirection_10m': get_list_median(stormData, 'winddirection_10m'),
        
        'avg_winddirection_100m': get_list_mean(stormData, 'winddirection_100m'),
        'min_winddirection_100m': get_list_min(stormData, 'winddirection_100m'),
        'max_winddirection_100m': get_list_max(stormData, 'winddirection_100m'),
        'med_winddirection_100m': get_list_median(stormData, 'winddirection_100m'),
        
        'avg_windgusts_10m': get_list_mean(stormData, 'windgusts_10m'),
        'min_windgusts_10m': get_list_min(stormData, 'windgusts_10m'),
        'max_windgusts_10m': get_list_max(stormData, 'windgusts_10m'),
        'med_windgusts_10m': get_list_median(stormData, 'windgusts_10m'),
        
        'avg_soil_temperature_0_to_7cm': get_list_mean(stormData, 'soil_temperature_0_to_7cm'),
        'min_soil_temperature_0_to_7cm': get_list_min(stormData, 'soil_temperature_0_to_7cm'),
        'max_soil_temperature_0_to_7cm': get_list_max(stormData, 'soil_temperature_0_to_7cm'),
        'med_soil_temperature_0_to_7cm': get_list_median(stormData, 'soil_temperature_0_to_7cm'),
        
        'avg_soil_temperature_7_to_28cm': get_list_mean(stormData, 'soil_temperature_7_to_28cm'),
        'min_soil_temperature_7_to_28cm': get_list_min(stormData, 'soil_temperature_7_to_28cm'),
        'max_soil_temperature_7_to_28cm': get_list_max(stormData, 'soil_temperature_7_to_28cm'),
        'med_soil_temperature_7_to_28cm': get_list_median(stormData, 'soil_temperature_7_to_28cm'),
        
        'avg_soil_temperature_28_to_100cm': get_list_mean(stormData, 'soil_temperature_28_to_100cm'),
        'min_soil_temperature_28_to_100cm': get_list_min(stormData, 'soil_temperature_28_to_100cm'),
        'max_soil_temperature_28_to_100cm': get_list_max(stormData, 'soil_temperature_28_to_100cm'),
        'med_soil_temperature_28_to_100cm': get_list_median(stormData, 'soil_temperature_28_to_100cm'),
        
        'avg_soil_temperature_100_to_255cm': get_list_mean(stormData, 'soil_temperature_100_to_255cm'),
        'min_soil_temperature_100_to_255cm': get_list_min(stormData, 'soil_temperature_100_to_255cm'),
        'max_soil_temperature_100_to_255cm': get_list_max(stormData, 'soil_temperature_100_to_255cm'),
        'med_soil_temperature_100_to_255cm': get_list_median(stormData, 'soil_temperature_100_to_255cm'),
        
        'avg_soil_moisture_0_to_7cm': get_list_mean(stormData, 'soil_moisture_0_to_7cm'),
        'min_soil_moisture_0_to_7cm': get_list_min(stormData, 'soil_moisture_0_to_7cm'),
        'max_soil_moisture_0_to_7cm': get_list_max(stormData, 'soil_moisture_0_to_7cm'),
        'med_soil_moisture_0_to_7cm': get_list_median(stormData, 'soil_moisture_0_to_7cm'),
        
        'avg_soil_moisture_7_to_28cm': get_list_mean(stormData, 'soil_moisture_7_to_28cm'),
        'min_soil_moisture_7_to_28cm': get_list_min(stormData, 'soil_moisture_7_to_28cm'),
        'max_soil_moisture_7_to_28cm': get_list_max(stormData, 'soil_moisture_7_to_28cm'),
        'med_soil_moisture_7_to_28cm': get_list_median(stormData, 'soil_moisture_7_to_28cm'),
        
        'avg_soil_moisture_28_to_100cm': get_list_mean(stormData, 'soil_moisture_28_to_100cm'),
        'min_soil_moisture_28_to_100cm': get_list_min(stormData, 'soil_moisture_28_to_100cm'),
        'max_soil_moisture_28_to_100cm': get_list_max(stormData, 'soil_moisture_28_to_100cm'),
        'med_soil_moisture_28_to_100cm': get_list_median(stormData, 'soil_moisture_28_to_100cm'),
        
        'avg_soil_moisture_100_to_255cm': get_list_mean(stormData, 'soil_moisture_100_to_255cm'),
        'min_soil_moisture_100_to_255cm': get_list_min(stormData, 'soil_moisture_100_to_255cm'),
        'max_soil_moisture_100_to_255cm': get_list_max(stormData, 'soil_moisture_100_to_255cm'),
        'med_soil_moisture_100_to_255cm': get_list_median(stormData, 'soil_moisture_100_to_255cm'),
                
        'target_cost': stormData['target_cost'][0]
        }
    return newRowData

In [29]:
storms = list(aggData.keys())
storms

['Georges',
 'Isidore',
 'Lili',
 'Ivan',
 'Dorian',
 'Allen',
 'Elsa',
 'Andrew',
 'Frances',
 'Charley',
 'Marilyn',
 'Dennis',
 'Maria',
 'Ian',
 'Hugo',
 'Isaias',
 'Sandy',
 'Florence',
 'Harvey',
 'Gloria',
 'Isabel',
 'Matthew',
 'Gustav',
 'Eta',
 'Fred',
 'Fran',
 'Laura',
 'Floyd',
 'Bonnie',
 'Isaac',
 'Irene',
 'Wilma',
 'Fiona',
 'Jeanne',
 'Irma',
 'Delta',
 'Ida',
 'Ike',
 'Zeta',
 'Dolly',
 'Michael',
 'Opal',
 'Elena',
 'Nicole',
 'Nicholas',
 'Rita',
 'Alberto',
 'Erin',
 'Katrina',
 'Juan',
 'Sally',
 'Bob',
 'Hanna',
 'Lee',
 'Alicia',
 'Allison',
 'Imelda']

In [30]:
data = []
for storm in storms: 
    data.append(generateNewRow(aggData[storm], storm))

In [32]:
aggDF = pd.DataFrame(data)
aggDF.shape

(57, 117)

In [33]:
aggDF.to_csv('./df_agg_57.csv', index = False)